In [ ]:
! pip install --upgrade pip
!pip install pystan==2.19.1.1 convertdate prophet

     |████████████████████████████████| 2.1 MB 31.0 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 20.1.1
    Uninstalling pip-20.1.1:
      Successfully uninstalled pip-20.1.1


In [ ]:
import pandas as pd
import os
import tqdm as tq
import string
import statistics as stats
import re
import datetime
import numpy as np
pd.set_option('display.max_columns', None) # allows to display all columns of a large dataframe
from prophet import Prophet
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from prophet.plot import plot_cross_validation_metric
import plotly.graph_objs as go

In [ ]:
msoa_lst = ['E02000001',
 'W02000335',
 'W02000336',
 'W02000337',
 'W02000338',
 'W02000339',
 'W02000340',
 'W02000341',
 'W02000342',
 'W02000343',
 'W02000344',
 'W02000345',
 'W02000346',
 'W02000347',
 'W02000348',
 'W02000349',
 'W02000350',
 'W02000351',
 'W02000352',
 'W02000353',
 'W02000354',
 'W02000355',
 'W02000356',
 'W02000357',
 'W02000358',
 'W02000359',
 'W02000360',
 'W02000361',
 'W02000362',
 'W02000363',
 'W02000364',
 'W02000365',
 'W02000366']

In [ ]:
df = pd.read_csv('/work/Prophet testing/crimes_and_features.csv')

In [ ]:
string = pd.read_csv('/work/Stringency Index/stringency.csv')
string['ds'] = pd.to_datetime(string['Month'])

#Defining types of lockdowns
no_lock = string[string['stringency_index'] == 0]['ds'].to_list()
med_lock = string[(string['stringency_index'] > 0) & (string['stringency_index'] < 50)]['ds'].to_list()
hard_lock = string[(string['stringency_index'] >= 50)]['ds'].to_list()

#Defining prophet 'holidays'
no_lockdown = pd.DataFrame({
    'holiday': 'no_lockdown',
    'ds': no_lock,
    'lower_window': 0,
    'upper_window': 0,
})

med_lockdown = pd.DataFrame({
    'holiday': 'med_lockdown',
    'ds': med_lock,
    'lower_window': 0,
    'upper_window': 0,
})

hard_lockdown = pd.DataFrame({
    'holiday': 'hard_lockdown',
    'ds': hard_lock,
    'lower_window': 0,
    'upper_window': 0,
})

holidays = pd.concat((no_lockdown, med_lockdown, hard_lockdown))

In [ ]:
df_forecast = pd.DataFrame()

In [ ]:
for msoa in msoa_lst:
    df2 = df[df['MSOA code'] == msoa]

    #This loops over every crime type in the dataset and makes seperate predicitons about the amount of that type
    #one year ahead of time, per month and prints a forecasting figure as result
    i = 3
    forecasts = []
    crime_types = []
    evaluations = []

    while i<=17: #replace 3 by once dataset implemented df.count(axis='columns')

        df_train2 = df2.iloc[:, [1, i]]
        crime_type = df_train2.columns.values[1]
        df_train2.columns.values[0] = "ds"
        df_train2.columns.values[1] = "y"
        df_train2.astype({'ds': 'datetime64[ns]'}).dtypes
        m = Prophet(holidays = holidays)
        m.fit(df_train2)

        future = m.make_future_dataframe(periods=12, freq='MS')

        forecast = m.predict(future)

        df_all = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
        df_all['Crime type'] = crime_type
        df_all['MSOA code'] = msoa
        
        df_2020 = df_all.loc[70:81] #previous year
        df_all = df_all.loc[82:] #predicted year

        df_all['past_yhat'] = df_2020['yhat'].tolist()
        df_all['past_yhat_lower'] = df_2020['yhat_lower'].tolist()
        df_all['past_yhat_upper'] = df_2020['yhat_upper'].tolist()

        df_all['change'] = ((df_all['yhat'] - df_all['past_yhat']) / df_all['past_yhat'])* 100 #overall change
        df_all['change_min'] = ((df_all['yhat_lower'] - df_all['past_yhat_lower']) / df_all['past_yhat_lower'])* 100 #lower change
        df_all['change_max'] = ((df_all['yhat_upper'] - df_all['past_yhat_upper']) / df_all['past_yhat_upper'])* 100 #upper change

        df_forecast = df_forecast.append(df_all)
        # Initial training period.
        #initial= 30*70
        #initial= str(initial)+' days'
        #Period length that we perform the cross validation for.
        #period = 30
        #period=str(period)+' days'
        #Horizon of prediction essentially for each fold.
        #horizon = 61
        #horizon=str(horizon)+' days'
        #fb_cv=cross_validation(m,initial=initial,period=period, 
        #horizon=horizon)
        # Performance Metrics of fb_cv
        #evaluation = performance_metrics(fb_cv, monthly =True, rolling_window=0.1)
        #evaluations.append(evaluation)

        i +=1

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/shared-libs/python3.7/py-core/lib/python3.7/site-pa

ValueError: Found infinity in column y.

In [ ]:
df_forecast.reset_index(inplace=True)

In [ ]:
df_forecast['District'] = 'City of London'
df_forecast.loc[180:, 'District'] = 'Gwent'
df_forecast = df_forecast.drop(['index', 'past_yhat', 'past_yhat_lower', 'past_yhat_upper'], axis = 1)
df_forecast = df_forecast.rename(columns={'ds': 'Month', 'yhat': 'prediction', 'yhat_lower': 'min', 'yhat_upper': 'max'})

In [ ]:
#df_forecast.to_csv('/work/final_pred3.csv')

In [ ]:
df_forecast

,Month,prediction,min,max,Crime type,MSOA code,change,change_min,change_max,District
0,2021-11-01,99.368048,67.235705,135.340802,Crime type_Anti-social behaviour,E02000001,25.985465,40.508476,21.538302,City of London
1,2021-12-01,91.826211,58.002916,122.999516,Crime type_Anti-social behaviour,E02000001,30.848553,64.975791,22.117851,City of London
2,2022-01-01,92.436646,58.316665,125.346675,Crime type_Anti-social behaviour,E02000001,43.207803,87.344734,26.679647,City of London
3,2022-02-01,88.408645,56.133321,123.785046,Crime type_Anti-social behaviour,E02000001,30.899288,81.809903,20.613550,City of London
4,2022-03-01,104.316427,71.254133,138.797857,Crime type_Anti-social behaviour,E02000001,49.208726,102.112395,36.718315,City of London
...,...,...,...,...,...,...,...,...,...,...
5935,2022-06-01,15.986031,1.363750,29.634735,Sum crimes,W02000366,-78.001607,-97.633863,-66.050254,Gwent
5936,2022-07-01,17.812082,2.828914,31.882579,Sum crimes,W02000366,-67.569072,-93.067625,-54.511222,Gwent
5937,2022-08-01,3.989886,-10.889581,18.687008,Sum crimes,W02000366,-91.281235,-134.065519,-68.459831,Gwent
5938,2022-09-01,8.611552,-5.257343,22.630522,Sum crimes,W02000366,-81.577033,-116.279341,-62.285899,Gwent


In [ ]:
# # Initial training period.
# initial= 30*70
# initial= str(initial)+' days'
# #Period length that we perform the cross validation for.
# period = 30
# period=str(period)+' days'
# #Horizon of prediction essentially for each fold.
# horizon = 61
# horizon=str(horizon)+' days'
# fb_cv=cross_validation(m,initial=initial,period=period, 
# horizon=horizon)
# # Performance Metrics of fb_cv
# evaluation = performance_metrics(fb_cv, monthly =True, rolling_window=0.1)

In [ ]:

#evaluation

NameError: name 'evaluation' is not defined

In [ ]:
#import itertools
#import numpy as np
#import pandas as pd

#param_grid = {  
 #   'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
  #  'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
   # 'seasonality_mode': ['additive', 'multiplicative'],
    #'holidays_prior_scale': [0.01, 0.1, 1.0, 10.0]
}

# Generate all combinations of parameters
#all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
#rmses = []  # Store the RMSEs for each params here

# Use cross validation to evaluate all parameters
#for params in tq.tqdm(all_params):
#    m = Prophet(**params, holidays = holidays).fit(df_train2)  # Fit model with given params
#    df_cv = cross_validation(m, horizon= horizon, parallel="processes") # add cutoffs?
#    df_p = performance_metrics(df_cv, rolling_window=0.1, monthly = True)
#    rmses.append(df_p['rmse'].values[0])

# Find the best parameters
#tuning_results = pd.DataFrame(all_params)
#tuning_results['rmse'] = rmses
#print(tuning_results)

SyntaxError: invalid syntax (<ipython-input-16-a56672029521>, line 10)

In [ ]:
#best_params = all_params[np.argmin(rmses)]
#print(best_params)
# {'changepoint_prior_scale': 0.1, 'seasonality_prior_scale': 0.01, 'seasonality_mode': 'additive', 'holidays_prior_scale': 0.01}

{'changepoint_prior_scale': 0.1, 'seasonality_prior_scale': 0.01, 'seasonality_mode': 'multiplicative', 'holidays_prior_scale': 1.0}


,Unnamed: 0,Month,prediction,min,max,Crime type,MSOA code,change,change_min,change_max,District
186,186,2022-05-01,-1.944928,-8.030248,4.429741,Crime type_Anti-social behaviour,W02000335,-107.618734,-141.974210,-85.963809,Gwent
187,187,2022-06-01,-4.103251,-10.246207,2.459907,Crime type_Anti-social behaviour,W02000335,-118.554678,-164.842078,-91.349428,Gwent
188,188,2022-07-01,-3.798575,-9.926806,2.361384,Crime type_Anti-social behaviour,W02000335,-145.185910,-531.501114,-83.834426,Gwent
189,189,2022-08-01,-6.465542,-12.221430,0.577586,Crime type_Anti-social behaviour,W02000335,-224.795923,901.661907,-95.105373,Gwent
190,190,2022-09-01,-8.058225,-14.361641,-1.448672,Crime type_Anti-social behaviour,W02000335,-256.888431,1414.845458,-112.560148,Gwent
...,...,...,...,...,...,...,...,...,...,...,...
5911,5911,2022-06-01,-2.012339,-5.364375,1.440638,Crime type_Vehicle crime,W02000366,-146.393736,-664.972852,-80.840932,Gwent
5912,5912,2022-07-01,-2.591207,-6.199585,0.790667,Crime type_Vehicle crime,W02000366,-178.701051,6144.318810,-88.575240,Gwent
5913,5913,2022-08-01,-4.349123,-7.689076,-1.021507,Crime type_Vehicle crime,W02000366,-229.587858,-13942.903920,-115.195511,Gwent
5914,5914,2022-09-01,-2.454382,-6.037004,1.099414,Crime type_Vehicle crime,W02000366,-162.756868,-1144.976928,-85.077533,Gwent


,Unnamed: 0,ds,yhat,yhat_lower,yhat_upper,Crime type,MSOA code,change,change_min,change_max,District
192,192,2021-11-01,-0.176038,-0.754366,0.401387,Crime type_Bicycle theft,W02000335,83.354183,15.068549,-10.442758,Gwent
193,193,2021-12-01,-0.070104,-0.666619,0.498352,Crime type_Bicycle theft,W02000335,-39.700543,-0.737698,15.561179,Gwent
194,194,2022-01-01,-0.158848,-0.729588,0.417194,Crime type_Bicycle theft,W02000335,1342.487144,35.980879,-19.577272,Gwent
195,195,2022-02-01,-0.141507,-0.729828,0.459320,Crime type_Bicycle theft,W02000335,-397.896738,42.484530,-21.967886,Gwent
198,198,2022-05-01,-0.261835,-0.780598,0.291458,Crime type_Bicycle theft,W02000335,95.988640,9.697339,-35.129800,Gwent
...,...,...,...,...,...,...,...,...,...,...,...
5805,5805,2022-08-01,-1.563905,-4.311234,1.132574,Crime type_Criminal damage and arson,W02000366,852.227737,53.407659,-56.845604,Gwent
5806,5806,2022-09-01,-1.758241,-4.459496,0.983628,Crime type_Criminal damage and arson,W02000366,123.498915,31.978045,-51.674209,Gwent
5882,5882,2022-01-01,-0.231334,-1.579743,1.100588,Crime type_Shoplifting,W02000366,-25.895624,-8.367659,-0.809685,Gwent
5896,5896,2022-03-01,-0.116777,-0.763562,0.540507,Crime type_Theft from the person,W02000366,434.056257,14.547566,-16.392679,Gwent


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=40789b9a-1c62-45b9-9d9c-b1a39ebe3dfd' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>